In [ ]:
# 2) Dataset scan + build data.yaml for pose (robust scan)
from pathlib import Path
import os, yaml
from glob import glob
from collections import Counter

root = Path('./dataset')
assert root.exists(), "Dataset folder './dataset' not found. Please scp/rsync it first."

# Find YOLO label .txt anywhere under labels/ (supports both layouts)
label_txts = []
label_txts += glob(str(root / 'labels' / '**' / '*.txt'), recursive=True)          # type-first: dataset/labels/train/*.txt
label_txts += glob(str(root / '*' / 'labels' / '**' / '*.txt'), recursive=True)    # split-first: dataset/train/labels/*.txt
label_txts = sorted(set(label_txts))
print(f"Found {len(label_txts)} label files under 'labels' folders")

if len(label_txts) == 0:
    # Last resort: search any .txt and warn
    label_txts = glob(str(root / '**' / '*.txt'), recursive=True)
    print(f"Fallback: found {len(label_txts)} .txt files total")
    assert len(label_txts)>0, "No label .txt files found anywhere in './dataset'. Expected YOLO-pose labels under .../labels/..."

# Robustly infer keypoints K by majority across many labels
kp_counter = Counter()
for i, fp in enumerate(label_txts[:2000]):  # sample up to 2000 files
    try:
        with open(fp, 'r') as f:
            for j, ln in enumerate(f):
                ln = ln.strip()
                if not ln:
                    continue
                parts = ln.split()
                # prefer full pose labels: cls cx cy w h + 3*K
                if len(parts) >= 8 and (len(parts) - 5) % 3 == 0:
                    Kcand = (len(parts) - 5) // 3
                    kp_counter[Kcand] += 1
                # stop early per file after a couple of lines
                if j >= 2:
                    break
    except Exception:
        continue

if not kp_counter:
    # Fallback: detect K from kpt-only (cls x1 y1 x2 y2 ...)
    kpt_only_counter = Counter()
    for i, fp in enumerate(label_txts[:2000]):
        try:
            with open(fp, 'r') as f:
                for j, ln in enumerate(f):
                    ln = ln.strip()
                    if not ln:
                        continue
                    parts = ln.split()
                    if len(parts) >= 3 and (len(parts) - 1) % 2 == 0:
                        Kcand = (len(parts) - 1) // 2
                        kpt_only_counter[Kcand] += 1
                    if j >= 2:
                        break
        except Exception:
            continue
    assert kpt_only_counter, 'Could not infer keypoints K from labels.'
    K = max(kpt_only_counter, key=kpt_only_counter.get)
else:
    K = max(kp_counter, key=kp_counter.get)

print(f"Inferred keypoints K (majority) = {K}")

# Heuristics to resolve images dirs for each split
def choose_images_dir(split: str):
    candidates = [
        root / 'images' / split,       # type-first
        root / split / 'images',       # split-first
    ]
    # derive from labels/<split>
    split_label_dirs = []
    for p in label_txts:
        p_norm = p.replace('\\', '/').lower()
        if f"/labels/{split}/" in p_norm:
            split_label_dirs.append(Path(p).parent)
    if split_label_dirs:
        ldir = split_label_dirs[0]
        candidates += [
            ldir.parent / 'images',                  # .../train/images
            ldir.parent.parent / 'images' / ldir.name # .../images/train
        ]
    for c in candidates:
        if c.exists():
            return str(c)
    return None

train_path = choose_images_dir('train')
val_path = choose_images_dir('val')
test_path = choose_images_dir('test')
print('Images dirs (raw):', {'train': train_path, 'val': val_path, 'test': test_path})

# Normalize to be RELATIVE to dataset root if possible, else absolute
root_abs = root.resolve()

def to_rel_or_abs(p):
    if not p:
        return None
    p_abs = Path(p)
    try:
        p_abs = p_abs.resolve()
    except Exception:
        p_abs = (root / p).resolve()
    try:
        return str(p_abs.relative_to(root_abs))
    except ValueError:
        return str(p_abs)

train_rel = to_rel_or_abs(train_path)
val_rel = to_rel_or_abs(val_path)
test_rel = to_rel_or_abs(test_path)
print('Images dirs (normalized):', {'train': train_rel, 'val': val_rel, 'test': test_rel})

assert train_rel and val_rel, (
    "Could not locate images/train and/or images/val. Ensure YOLO structure like:\n"
    "- dataset/images/train & dataset/labels/train\n"
    "- dataset/train/images & dataset/train/labels\n"
)

# Build YAML for pose
names = ['plate']
skeleton = [[0,1],[1,2],[2,3],[3,0]] if K==4 else []
flip_idx = list(range(K))

data = {
    'path': str(root_abs),
    'train': train_rel,
    'val': val_rel,
    'test': test_rel,
    'names': names,
    'kpt_shape': [K, 3],
    'skeleton': skeleton,
    'flip_idx': flip_idx,
}

yaml_path = root/'data.yaml'
with open(yaml_path, 'w') as f:
    yaml.safe_dump(data, f, sort_keys=False)

print('Wrote', yaml_path)
print(yaml.safe_dump(data, sort_keys=False))

# Clear label caches so Ultralytics rebuilds with the corrected kpt_shape
for cache_name in ['train.cache', 'val.cache']:
    cp = root / 'labels' / cache_name
    if cp.exists():
        try:
            os.remove(cp)
            print('Removed cache:', cp)
        except Exception as e:
            print('Failed to remove cache', cp, e)


In [ ]:
# 2.5) Quick audit (10 files) + auto-fix labels to YOLOv8-Pose format if needed
from pathlib import Path
import os, shutil, glob

labels_root = Path('./dataset/labels')
assert labels_root.exists(), 'labels folder not found'

# Sample ~10 label files from val to inspect
samples = []
for split in ['val', 'train']:
    cand = sorted(glob.glob(str(labels_root / split / '*.txt')))
    if cand:
        samples += cand[:max(0, 10 - len(samples))]
    if len(samples) >= 10:
        break

print('Sampling files:', len(samples))

# Inspect helper
def classify_format(tokens):
    # tokens: list[str]
    n = len(tokens)
    if n >= 6 and (n - 5) % 3 == 0:
        return 'yolo_pose_full'  # cls x y w h x1 y1 v1 ...
    if n >= 3 and (n - 1) % 2 == 0:
        return 'kpt_only'        # cls x1 y1 x2 y2 ... (no bbox, no v)
    return 'unknown'

bad_examples = []
for fp in samples:
    with open(fp, 'r') as f:
        lines = [ln.strip() for ln in f.readlines() if ln.strip()]
    for ln in lines[:2]:  # show at most 2 lines/file
        toks = ln.split()
        fmt = classify_format(toks)
        if fmt != 'yolo_pose_full':
            bad_examples.append((fp, ln, fmt))
        else:
            # quick sanity: visibility values should be integers 0/1/2
            try:
                # v1 begins at index 7 (0-based): [cls,cx,cy,w,h,x1,y1,v1,x2,y2,v2,...]
                vis_vals = [float(v) for v in toks[7::3]]
                if not all(v in (0, 1, 2) for v in vis_vals):
                    bad_examples.append((fp, ln, 'invalid_visibility'))
            except Exception:
                pass

print('Found non-compliant examples (up to 10 shown):')
for i, (fp, ln, fmt) in enumerate(bad_examples[:10], 1):
    print(f'{i:02d}. {fp} | fmt={fmt} | line="{ln[:120]}"')

# If most samples are kpt_only, auto-convert the whole dataset in-place (with backup)
needs_convert = any(fmt == 'kpt_only' for _, _, fmt in bad_examples) and not any(fmt == 'yolo_pose_full' for _, _, fmt in bad_examples)
print('Needs conversion:', needs_convert)


def convert_file_in_place(txt_path: Path, backup_dir: Path, expected_k: int | None = None) -> tuple[int, int]:
    """Convert kpt-only lines to YOLOv8-pose full format. Returns (ok_lines, total_lines)."""
    with open(txt_path, 'r') as f:
        lines = [ln.strip() for ln in f.readlines() if ln.strip()]

    out_lines = []
    ok, total = 0, 0
    for ln in lines:
        total += 1
        toks = ln.split()
        fmt = classify_format(toks)
        if fmt == 'yolo_pose_full':
            out_lines.append(ln)
            ok += 1
            continue
        if fmt != 'kpt_only':
            # skip malformed lines
            continue
        try:
            cls = int(float(toks[0]))
        except Exception:
            continue
        vals = list(map(float, toks[1:]))
        if len(vals) % 2 != 0:
            continue
        K = len(vals) // 2
        if expected_k is not None and K != expected_k:
            # inconsistent K
            continue
        xs = vals[0::2]
        ys = vals[1::2]
        # clamp to [0,1]
        xs = [min(1.0, max(0.0, x)) for x in xs]
        ys = [min(1.0, max(0.0, y)) for y in ys]
        x1, x2 = min(xs), max(xs)
        y1, y2 = min(ys), max(ys)
        w = max(x2 - x1, 1e-6)
        h = max(y2 - y1, 1e-6)
        cx = (x1 + x2) / 2.0
        cy = (y1 + y2) / 2.0
        # visibility: mark as 2 (labeled & visible)
        kv = []
        for i in range(K):
            kv += [xs[i], ys[i], 2]
        new_toks = [str(cls), f'{cx:.6f}', f'{cy:.6f}', f'{w:.6f}', f'{h:.6f}'] + [f'{v:.6f}' if isinstance(v, float) else str(v) for v in kv]
        out_lines.append(' '.join(new_toks))
        ok += 1

    if ok:
        # backup original once
        backup_path = backup_dir / txt_path.name
        if not backup_path.exists():
            shutil.copy2(txt_path, backup_path)
        with open(txt_path, 'w') as f:
            f.write('\n'.join(out_lines) + ('\n' if out_lines else ''))
    return ok, total

if needs_convert:
    backup_root = labels_root.parent / 'labels_backup_before_pose_fix'
    backup_root.mkdir(exist_ok=True)
    sum_ok = 0
    sum_total = 0
    # Infer K from first bad sample
    K_guess = None
    for _, ln, fmt in bad_examples:
        if fmt == 'kpt_only':
            toks = ln.split()
            K_guess = (len(toks) - 1) // 2
            break
    print('Assumed K from samples =', K_guess)

    for split in ['train', 'val']:
        split_dir = labels_root / split
        if not split_dir.exists():
            continue
        backup_dir = backup_root / split
        backup_dir.mkdir(parents=True, exist_ok=True)
        files = sorted(split_dir.glob('*.txt'))
        for i, fp in enumerate(files, 1):
            ok, total = convert_file_in_place(fp, backup_dir, expected_k=K_guess)
            sum_ok += ok
            sum_total += total
            if i % 500 == 0:
                print(f'Converted {i}/{len(files)} files...')
    print(f'Converted lines: {sum_ok}/{sum_total}. Backup at: {backup_root}')

    # Clear caches so Ultralytics rebuilds with new labels
    for cache_name in ['train.cache', 'val.cache']:
        cp = labels_root / cache_name
        if cp.exists():
            try:
                os.remove(cp)
                print('Removed cache:', cp)
            except Exception as e:
                print('Failed to remove cache', cp, e)
else:
    print('Labels appear to already be in YOLOv8-pose format for sampled files. No conversion applied.')


In [ ]:
# 2.6) Normalize class IDs to single class (0) and clear caches
from pathlib import Path
import os, glob

labels_root = Path('./dataset/labels')
assert labels_root.exists(), 'labels folder not found'

changed_lines = 0
changed_files = 0

def normalize_classes(txt_path: Path) -> tuple[int, bool]:
    with open(txt_path, 'r') as f:
        lines = [ln.rstrip('\n') for ln in f.readlines()]
    out = []
    changed = False
    count = 0
    for ln in lines:
        if not ln.strip():
            out.append(ln)
            continue
        toks = ln.split()
        try:
            cls = int(float(toks[0]))
        except Exception:
            out.append(ln)
            continue
        if cls != 0:
            toks[0] = '0'
            changed = True
            count += 1
        out.append(' '.join(toks))
    if changed:
        with open(txt_path, 'w') as f:
            f.write('\n'.join(out) + ('\n' if out and out[-1] != '' else ''))
    return count, changed

for split in ['train', 'val']:
    files = sorted((labels_root / split).glob('*.txt')) if (labels_root / split).exists() else []
    for fp in files:
        cnt, ch = normalize_classes(fp)
        changed_lines += cnt
        changed_files += int(ch)

print(f'Normalized class IDs in {changed_files} files, {changed_lines} lines changed to class 0.')

# Clear caches again
for cache_name in ['train.cache', 'val.cache']:
    cp = labels_root / cache_name
    if cp.exists():
        try:
            os.remove(cp)
            print('Removed cache:', cp)
        except Exception as e:
            print('Failed to remove cache', cp, e)


In [4]:
# 3) Train
from ultralytics import YOLO
from pathlib import Path
import torch
import os, glob

# Disable cloud sync/telemetry to reduce surprise downloads
try:
    from ultralytics.utils import SETTINGS
    SETTINGS.update({'sync': False})
except Exception:
    pass

yaml_path = Path('./dataset/data.yaml')
assert yaml_path.exists(), 'data.yaml not found; run the dataset YAML cell.'

# Remove unrelated YOLOv11/YOLO11 detection weights to avoid accidental selection
for pat in ['yolo11*.pt', 'yolov11*.pt']:
    for p in glob.glob(pat):
        try:
            os.remove(p)
            print('Removed unrelated weight:', p)
        except Exception as e:
            print('Could not remove', p, e)

model_name = 'yolov8n-pose.pt'  # nano pose model
model = YOLO(model_name)
print('Using base weights:', model_name, '| task:', getattr(model, 'task', None))

# Training options (up to 200 epochs with early stopping)
train_args = dict(
    data=str(yaml_path),
    imgsz=768,        # bigger image size helps pose accuracy
    epochs=200,       # train up to 200 epochs
    patience=30,      # early stop if no val improvement for 30 epochs
    batch=4,          # adjust if you hit OOM
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=2,        # keep low for laptops
    project='runs',
    name='pose_plate',
    exist_ok=True,
    pretrained=False,  # avoid triggering any default weight downloads
    cache=False,
    plots=True,       # save results.png and curves
    save_period=20,   # checkpoint every 20 epochs
    seed=42,
 )

results = model.train(**train_args)
print('Training done. Best:', results)

Using base weights: yolov8n-pose.pt | task: pose
New https://pypi.org/project/ultralytics/8.3.195 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.192 🚀 Python-3.13.7 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3768MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=

In [ ]:
# 4) Validate & quick inference
from glob import glob
import os, yaml
import cv2, numpy as np
from ultralytics import YOLO

# locate last run more robustly
pose_runs = sorted(glob('runs/pose_plate/*')) if os.path.exists('runs/pose_plate') else []
run_dir = pose_runs[-1] if pose_runs else None
print('Last run:', run_dir)

# Load model from best/last if available, otherwise prefer local yolov8 pose weights
ckpt = None
if run_dir:
    for w in ['weights/best.pt', 'weights/last.pt']:
        p = os.path.join(run_dir, w)
        if os.path.exists(p):
            ckpt = p
            break

fallback = 'yolov8n-pose.pt'
print('Loading weights:', ckpt or fallback)
model = YOLO(ckpt or fallback)
print('Model task:', getattr(model, 'task', None))

# read data.yaml to find val images path
with open('./dataset/data.yaml', 'r') as f:
    data_cfg = yaml.safe_load(f)
val_path = data_cfg.get('val')
base_path = data_cfg.get('path', '.')
val_dir = os.path.join(base_path, val_path) if val_path else './dataset/val/images'

# pick a few images
val_imgs = []
for pat in ['*.jpg', '*.jpeg', '*.png', '*.*']:
    val_imgs = glob(os.path.join(val_dir, pat))
    if val_imgs:
        break
val_imgs = val_imgs[:6]
print('Previewing', len(val_imgs), 'images from', val_dir)


def _to_numpy(x):
    try:
        import torch
        if isinstance(x, np.ndarray):
            return x
        if isinstance(x, torch.Tensor):
            return x.detach().cpu().numpy()
    except Exception:
        pass
    return np.asarray(x)


def _order_points_four(pts: np.ndarray) -> np.ndarray:
    pts = np.asarray(pts, dtype='float32')
    if pts.shape[0] != 4:
        return pts
    s = pts.sum(axis=1)
    diff = pts[:, 1] - pts[:, 0]
    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]
    return np.array([tl, tr, br, bl], dtype='float32')


for imgp in val_imgs:
    im = cv2.imread(imgp)
    res = model.predict(source=im, imgsz=640, conf=0.5, iou=0.6, classes=[0], verbose=False, max_det=50)[0]

    # Draw boxes
    if res.boxes is not None and len(res.boxes) > 0:
        for b in res.boxes:
            x1,y1,x2,y2 = b.xyxy[0].int().cpu().tolist()
            cv2.rectangle(im, (x1,y1), (x2,y2), (0,255,0), 2)

    # Draw keypoints with CPU conversion and polygon
    if getattr(res, 'keypoints', None) is not None and res.keypoints.xy is not None:
        kxy = _to_numpy(res.keypoints.xy)
        kcf = _to_numpy(getattr(res.keypoints, 'conf', None)) if getattr(res.keypoints, 'conf', None) is not None else None
        for i in range(kxy.shape[0]):
            pts = kxy[i]
            if pts is None or pts.shape[0] < 4:
                continue
            mask = np.ones((pts.shape[0],), dtype=bool)
            if kcf is not None:
                mask = kcf[i] >= 0.1
            pts_vis = pts[mask]
            if pts_vis.shape[0] >= 4:
                pts4 = pts_vis[:4]
                ordered = _order_points_four(pts4)
                poly = ordered.reshape((-1,1,2)).astype(int)
                cv2.polylines(im, [poly], isClosed=True, color=(0,0,255), thickness=2)
                for px, py in ordered:
                    cv2.circle(im, (int(px), int(py)), 4, (0,255,255), -1)
            else:
                for px, py in pts[:4]:
                    cv2.circle(im, (int(px), int(py)), 4, (0,255,255), -1)

    cv2.imshow('preview', im)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
